<a href="https://colab.research.google.com/github/maleepicface/brianhelenfakenews/blob/main/URL_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install whois
!pip install pyquery
!pip install interruptingcow

import whois
from datetime import datetime, timezone
import math
import pandas as pd
import numpy as np
from pyquery import PyQuery
from requests import get
import datetime
import math
import pandas as pd
import numpy as np
import whois
from tqdm import tqdm
from interruptingcow import timeout
import os
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.preprocessing import MinMaxScaler
from keras.utils import np_utils


  Created wheel for whois: filename=whois-0.9.13-py3-none-any.whl size=9609 sha256=45dbe4c1b712cd862c5d601348e777a7a246f655a14eb6c219a2407316d9c3fb
  Stored in directory: /root/.cache/pip/wheels/f5/85/a6/2ac688355ea1a85b0a2b9c0243012f3021333b6c65b7305d86
Successfully built whois
  Created wheel for interruptingcow: filename=interruptingcow-0.8-py3-none-any.whl size=5822 sha256=189fa2158bbb6bc72bd9631394847f8e7cf0a21849480402e7eaaa75044ddf5e
  Stored in directory: /root/.cache/pip/wheels/fe/83/45/99297454c39c203890ab65c08db879940862778921cc61a416
Successfully built interruptingcow


In [ ]:
class UrlFeaturizer(object):
  def __init__(self, url):
    self.url = url
    self.domain = url.split('//')[-1].split('/')[0]
    self.today = datetime.datetime.now().replace(tzinfo=None)

    try:
      self.whois = whois.query(self.domain).__dict__
    except:
      self.whois = None
    
    try:
      self.response = get(self.url)
      self.pq = PyQuery(self.response.text)
    except:
      self.response = None
      self.pq = None

  def entropy(self):
        string = self.url.strip()
        prob = [float(string.count(c)) / len(string) for c in dict.fromkeys(list(string))]
        entropy = sum([(p * math.log(p) / math.log(2.0)) for p in prob])
        return entropy

  def ip(self):
        string = self.url
        flag = False
        if ("." in string):
            elements_array = string.strip().split(".")
            if(len(elements_array) == 4):
                for i in elements_array:
                    if (i.isnumeric() and int(i)>=0 and int(i)<=255):
                        flag=True
                    else:
                        flag=False
                        break
        if flag:
            return 1 
        else:
            return 0

  def numDigits(self):
        digits = [i for i in self.url if i.isdigit()]
        return len(digits)

  def urlLength(self):
      return len(self.url)

  def numParameters(self):
      params = self.url.split('&')
      return len(params) - 1

  def numFragments(self):
      fragments = self.url.split('#')
      return len(fragments) - 1

  def numSubDomains(self):
      subdomains = self.url.split('http')[-1].split('//')[-1].split('/')
      return len(subdomains)-1

  def domainExtension(self):
      ext = self.url.split('.')[-1].split('/')[0]
      return ext

  ## URL domain features
  def hasHttp(self):
      return 'http:' in self.url

  def hasHttps(self):
      return 'https:' in self.url

  def daysSinceRegistration(self):
      if self.whois and self.whois['creation_date']:
          diff = self.today - self.whois['creation_date'].replace(tzinfo=None)
          diff = str(diff).split(' days')[0]
          return diff
      else:
          return 0

  def daysSinceExpiration(self):
      if self.whois and self.whois['expiration_date']:
          diff = self.whois['expiration_date'].replace(tzinfo=None) - self.today
          diff = str(diff).split(' days')[0]
          return diff
      else:
          return 0
  
    ## URL Page Features
  def bodyLength(self):
      if self.pq is not None:
          return len(self.pq('html').text()) if self.urlIsLive else 0
      else:
          return 0

  def numTitles(self):
      if self.pq is not None:
          titles = ['h{}'.format(i) for i in range(7)]
          titles = [self.pq(i).items() for i in titles]
          return len([item for s in titles for item in s])
      else:
          return 0

  def numImages(self):
      if self.pq is not None:
          return len([i for i in self.pq('img').items()])
      else:
          return 0

  def numLinks(self):
      if self.pq is not None:
          return len([i for i in self.pq('a').items()])
      else:
          return 0

  def scriptLength(self):
      if self.pq is not None:
          return len(self.pq('script').text())
      else:
          return 0

  def specialCharacters(self):
      if self.pq is not None:
          bodyText = self.pq('html').text()
          schars = [i for i in bodyText if not i.isdigit() and not i.isalpha()]
          return len(schars)
      else:
          return 0

  def scriptToSpecialCharsRatio(self):
      v = self.specialCharacters()
      if self.pq is not None and v!=0:
          sscr = self.scriptLength()/v
      else:
          sscr = 0
      return sscr

  def scriptTobodyRatio(self):
      v = self.bodyLength()
      if self.pq is not None and v!=0:
          sbr = self.scriptLength()/v
      else:
          sbr = 0
      return sbr

  def bodyToSpecialCharRatio(self):
      v = self.bodyLength()
      if self.pq is not None and v!=0:
          bscr = self.specialCharacters()/v
      else:
          bscr = 0
      return bscr

  def urlIsLive(self):
      return self.response == 200

  def run(self):
      data = {}
      data['entropy'] = self.entropy()
      data['numDigits'] = self.numDigits()
      data['urlLength'] = self.urlLength()
      data['numParams'] = self.numParameters()
      data['hasHttp'] = self.hasHttp()
      data['hasHttps'] = self.hasHttps()
      data['urlIsLive'] = self.urlIsLive()
      data['bodyLength'] = self.bodyLength()
      data['numTitles'] = self.numTitles()
      data['numImages'] = self.numImages()
      data['numLinks'] = self.numLinks()
      data['scriptLength'] = self.scriptLength()
      data['specialChars'] = self.specialCharacters()
      data['ext'] = self.domainExtension()
      data['dsr'] = self.daysSinceRegistration()
      data['dse'] = self.daysSinceExpiration()
      data['sscr'] = self.scriptToSpecialCharsRatio()
      data['sbr'] = self.scriptTobodyRatio()
      data['bscr'] = self.bodyToSpecialCharRatio()
      data['num_%20'] = self.url.count("%20")
      data['num_@'] = self.url.count("@")
      data['has_ip'] = self.ip()
  
      return data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# 4/1AX4XfWhh6dp68rly3sGO-2h9L6c1s5Rkd0YKRpyUByYsNfHbzH1OSekZvTQ

Mounted at /content/drive


In [ ]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4508194595052697864
xla_global_id: -1
]


In [ ]:
l = ['phishing_features.csv']

emp = UrlFeaturizer("").run().keys()

A = pd.DataFrame(columns = emp)
t=[]
for j in l:
    print(j)
    d = pd.read_csv("/content/drive/MyDrive/phishing_features.csv")
    #d=pd.read_csv(j,header=None).to_numpy().flatten()
    for i in tqdm(d):
        try: 
            with timeout(30, exception = RuntimeError):  
                temp=UrlFeaturizer(i).run()
                temp["File"]=j.split(".")[0]
                t.append(temp)
        except RuntimeError: 
            pass 
A=A.append(t)
os.chdir('../')
A.to_csv("features.csv")

phishing_features.csv


  0%|          | 24/9965 [00:00<00:03, 2682.21it/s]


In [ ]:
#os.chdir("../")
#os.chdir("/content/drive/MyDrive/phishing_features.csv")

# In[3]:


def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


# In[4]:


data = pd.read_csv("/content/drive/MyDrive/phishing_features.csv")


# In[5]:


data.drop(columns='Unnamed: 0',inplace=True)
data.replace(True,1,inplace = True)
data.replace(False,0,inplace = True)
y = data["File"]


# In[6]:


data = data.drop(columns = "File")


# In[7]:


encoder = LabelEncoder()
Y = encoder.fit_transform(y)


# In[8]:


scaler = MinMaxScaler(feature_range=(0, 1))
#X = scaler.fit_transform(data)
#data = data.astype(float)
X = pd.DataFrame(X)


# In[9]:


from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Masking, Embedding

input_dim = len(data.columns)
model = Sequential()
model.add(Dense(256, input_dim = input_dim , activation = 'relu'))
model.add(Dense(128, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(16, activation = 'relu'))
model.add(Dense(5, activation = 'softmax'))

model.compile(loss = 'categorical_crossentropy' , optimizer = 'adam' , metrics = ['accuracy',f1_m,precision_m, recall_m] )


# In[10]:


X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=42)


# In[11]:


model.fit(X_train,np_utils.to_categorical(y_train),epochs = 50,validation_split=0.3, batch_size = 128)


# In[12]:


y_pred = model.predict(X_test)


# In[13]:


predicted = np.argmax(y_pred, axis=1)


# In[14]:


from sklearn.metrics import classification_report
print(accuracy_score(y_test,predicted))


# In[15]:


target_names = ['Benign','Defacement','Malware','Phishing','Spam']
print(classification_report(y_test, predicted, target_names=target_names))


# In[16]:


os.chdir("../")
os.chdir("models")
model.save("Model_v1.h5")
np.save('lblenc.npy', encoder.classes_)
scalerfile = 'scaler.sav'
pickle.dump(scaler, open(scalerfile, 'wb'))

NameError: ignored